In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd

In [9]:
!ls scala_test/out/artifacts/scala_test_jar/

scala_test.jar


In [17]:
spark = SparkSession.builder.appName('my_test').config('spark.jars', 'scala_test/out/artifacts/scala_test_jar/scala_test.jar').getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/jingmin1987/hadoop/spark-3.3.0/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/jingmin1987/hadoop/hadoop-3.3.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


22/07/14 22:06:58 WARN Utils: Your hostname, JazyJojo resolves to a loopback address: 127.0.1.1; using 192.168.1.155 instead (on interface wifi0)
22/07/14 22:06:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/14 22:06:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [32]:
from py4j.java_gateway import java_import, JavaGateway

In [44]:
gateway = JavaGateway()

In [45]:
gateway.entry_point.df()

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:25333)

In [47]:
spark._sc._gateway.jvm

In [48]:
spark._jvm

In [49]:
spark.sparkContext._gateway.jvm

In [22]:
java_import(spark._jvm, 'test.Hello')

In [31]:
spark._jvm.Hello() # This throws an error because it was created as a Scala singleton object, not a java class so there's no constructor

In [4]:
df_pd = pd.DataFrame({'id': ['a', 'b', 'a'], 'val': [1, 2, 3]})

In [5]:
df = spark.createDataFrame(df_pd)

In [6]:
df.show()

+---+---+
| id|val|
+---+---+
|  a|  1|
|  b|  2|
|  a|  3|
+---+---+



In [9]:
df \
    .groupBy('id') \
    .agg(F.sum('val').alias('sum_val'),
         F.mean('val').alias('mean_val')) \
    .toPandas()

,id,sum_val,mean_val
0,a,4,2.0
1,b,2,2.0
